In [1]:
from TrainDataPreprocess import ProcessData
from ModelSet import NERModel, CategoriesClassifier, PolaritiesClassifier
from GetData import GetData

train_reviews_path = '../data/Train_reviews.csv'
train_labels_path = '../data/Train_labels.csv'
test_reviews_path = '../data/Test_reviews.csv'

/Users/zhaoliang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#预处理数据
process_data = ProcessData(train_reviews_path, train_labels_path)
process_data.sava_data()

#获取训练数据和测试数据
get_data_obj = GetData('char')
chunk2id, id2chunk, vocab2id, id2vocab, ner_data, ner_label = get_data_obj.get_ner_train_data()
content_data, aspect_data, opinion_data, category_label, id2categories = get_data_obj.get_categories_train_data()
content_data, aspect_data, opinion_data, polarity_label, id2polarities = get_data_obj.get_polarities_train_data()
ids, data, sents = get_data_obj.get_test_ner_data(test_reviews_path, vocab2id)

Building prefix dict from the default dictionary ...
I0823 17:09:58.628165 4511393216 __init__.py:111] Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/x8/4yf3l2h548b6w9d_ym2c1lvr0000gp/T/jieba.cache
I0823 17:09:58.629618 4511393216 __init__.py:131] Loading model from cache /var/folders/x8/4yf3l2h548b6w9d_ym2c1lvr0000gp/T/jieba.cache


数据预处理...
数据表整合完成...


Loading model cost 0.648 seconds.
I0823 17:09:59.277382 4511393216 __init__.py:163] Loading model cost 0.648 seconds.
Prefix dict has been built succesfully.
I0823 17:09:59.278607 4511393216 __init__.py:164] Prefix dict has been built succesfully.


字典生成与存储...
字典生成与存储完毕
数据生成与存储...
数据生成与存储完毕！


In [3]:
max_sent_len = 0
for item in content_data:
    if len(item) > max_sent_len:
        max_sent_len = len(item)
for item in data:
    if len(item) > max_sent_len:
        max_sent_len = len(item)

In [4]:
ner = NERModel(maxlen = max_sent_len, word_dict_size = len(vocab2id), word_vec_size = 128, class_label_count = len(chunk2id))
ner.train(ner_data, ner_label )
ner_result = ner.predict(data, id2chunk)

W0823 17:10:07.805237 4511393216 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0823 17:10:07.819478 4511393216 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0823 17:10:07.830490 4511393216 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0823 17:10:08.379146 4511393216 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_w

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 72)                0         
_________________________________________________________________
embedding_layer (Embedding)  (None, 72, 128)           165120    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 72, 64)            41216     
_________________________________________________________________
dropout_1 (Dropout)          (None, 72, 64)            0         
_________________________________________________________________
crf_1 (CRF)                  (None, 72, 5)             360       
Total params: 206,696
Trainable params: 206,696
Non-trainable params: 0
_________________________________________________________________


W0823 17:10:10.606213 4511393216 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 2906 samples, validate on 323 samples
Epoch 1/500
2906/2906 [==============================] - 10s 4ms/step - loss: 0.3832 - crf_viterbi_accuracy: 0.9033 - val_loss: 0.2276 - val_crf_viterbi_accuracy: 0.9080
Epoch 2/500
2906/2906 [==============================] - 9s 3ms/step - loss: 0.1850 - crf_viterbi_accuracy: 0.9260 - val_loss: 0.1410 - val_crf_viterbi_accuracy: 0.9571
Epoch 3/500
2906/2906 [==============================] - 8s 3ms/step - loss: 0.1222 - crf_viterbi_accuracy: 0.9586 - val_loss: 0.1111 - val_crf_viterbi_accuracy: 0.9627
Epoch 4/500
2906/2906 [==============================] - 8s 3ms/step - loss: 0.0988 - crf_viterbi_accuracy: 0.9645 - val_loss: 0.0967 - val_crf_viterbi_accuracy: 0.9662
Epoch 5/500
2906/2906 [==============================] - 8s 3ms/step - loss: 0.0840 - crf_viterbi_accuracy: 0.9707 - val_loss: 0.0857 - val_crf_viterbi_accuracy: 0.9714
Epoch 6/500
2906/2906 [==============================] - 8s 3ms/step - loss: 0.0723 - crf_viterbi_accuracy:

In [5]:
from GenSubmit import GenSubmit
gs = GenSubmit(list(process_data.train_reviews['Reviews']) + sents, id2vocab)

In [6]:
ner_table = gs.get_ner_result(ids, sents, data, ner_result)
ret_content_arr, ret_aspect_arr, ret_opinion_arr = get_data_obj.get_test_category_polarity_data(ner_table, vocab2id)

In [7]:
aspect_max_len = 0
for item in aspect_data:
    if len(item) > aspect_max_len:
        aspect_max_len = len(item)
for item in ret_aspect_arr:
    if len(item) > aspect_max_len:
        aspect_max_len = len(item)
        
option_max_len = 0
for item in opinion_data:
    if len(item) > option_max_len:
        option_max_len = len(item)
for item in ret_opinion_arr:
    if len(item) > option_max_len:
        option_max_len = len(item)

In [9]:
cc = CategoriesClassifier(content_max_len = max_sent_len, aspect_max_len=aspect_max_len, option_max_len=option_max_len, categories_num=len(id2categories), word_dict_size=len(vocab2id), word_vec_size=128)
cc.train(content_data, aspect_data, opinion_data, category_label)


/Users/zhaoliang/gitProject/zhijiang/code/ModelSet.py:175: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=50, kernel_size=5, padding="valid")`
  conv_layer = Conv1D(nb_filter=50, filter_length=(2 * half_window_size + 1), border_mode='valid')
W0823 17:13:32.377467 4511393216 deprecation_wrapper.py:119] From /Users/zhaoliang/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

/Users/zhaoliang/gitProject/zhijiang/code/ModelSet.py:188: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  model = Model(input=[content_input_layer, aspect_input_layer, option_input_layer], output=output)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
content_input_layer (InputLayer (None, 72)           0                                            
__________________________________________________________________________________________________
aspect_input_layer (InputLayer) (None, 8)            0                                            
__________________________________________________________________________________________________
option_input_layer (InputLayer) (None, 12)           0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     multiple             165120      content_input_layer[0][0]        
                                                                 aspect_input_layer[0][0]         
          

In [10]:
pc = PolaritiesClassifier(content_max_len = max_sent_len, aspect_max_len=aspect_max_len, option_max_len=option_max_len, word_dict_size=len(vocab2id), word_vec_size=128)
pc.train(content_data, aspect_data, opinion_data, polarity_label)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
content_input_layer (InputLayer (None, 72)           0                                            
__________________________________________________________________________________________________
aspect_input_layer (InputLayer) (None, 8)            0                                            
__________________________________________________________________________________________________
option_input_layer (InputLayer) (None, 12)           0                                            
__________________________________________________________________________________________________
embedding_layer (Embedding)     multiple             165120      content_input_layer[0][0]        
                                                                 aspect_input_layer[0][0]         
          

/Users/zhaoliang/gitProject/zhijiang/code/ModelSet.py:99: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=50, kernel_size=5, padding="valid")`
  conv_layer = Conv1D(nb_filter=50, filter_length=(2 * half_window_size + 1), border_mode='valid')
/Users/zhaoliang/gitProject/zhijiang/code/ModelSet.py:112: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  model = Model(input=[content_input_layer, aspect_input_layer, option_input_layer], output=output)


Train on 5969 samples, validate on 664 samples
Epoch 1/500
5969/5969 [==============================] - 3s 491us/step - loss: 0.2331 - acc: 0.9201 - val_loss: 0.1305 - val_acc: 0.9488
Epoch 2/500
5969/5969 [==============================] - 2s 373us/step - loss: 0.1083 - acc: 0.9611 - val_loss: 0.1163 - val_acc: 0.9578
Epoch 3/500
5969/5969 [==============================] - 2s 374us/step - loss: 0.0812 - acc: 0.9702 - val_loss: 0.1123 - val_acc: 0.9563
Epoch 4/500
5969/5969 [==============================] - 2s 370us/step - loss: 0.0622 - acc: 0.9784 - val_loss: 0.1115 - val_acc: 0.9503
Epoch 5/500
5969/5969 [==============================] - 2s 376us/step - loss: 0.0463 - acc: 0.9839 - val_loss: 0.1292 - val_acc: 0.9518
Epoch 6/500
5969/5969 [==============================] - 2s 376us/step - loss: 0.0325 - acc: 0.9905 - val_loss: 0.1664 - val_acc: 0.9458
Epoch 7/500
5969/5969 [==============================] - 2s 377us/step - loss: 0.0254 - acc: 0.9913 - val_loss: 0.1552 - val_acc: 0

In [11]:
ret_aspect_arr[0]

[0]

In [12]:
cc_result = cc.predict(ret_content_arr, ret_aspect_arr, ret_aspect_arr, id2categories)
pc_result = pc.predict(ret_content_arr, ret_aspect_arr, ret_opinion_arr)
ner_table['Categories'] = cc_result
ner_table['Polarities'] = pc_result
final_table = ner_table[['id', 'AspectTerms', 'OpinionTerms', 'Categories', 'Polarities']]
final_table.to_csv('../result/2019-08-23.csv', header=None, index=False)

In [13]:
final_table.head()

,id,AspectTerms,OpinionTerms,Categories,Polarities
0,1,_,最近太忙,尺寸,负面
1,1,_,是正品,尺寸,负面
2,1,_,白嫩,尺寸,负面
3,1,_,不错,尺寸,正面
4,2,颜色,_,功效,正面
